In [1]:
import sys
sys.path.append("../..")

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
# environments

from environment.deepqlearning.obstacle_avoidance_env import ObstacleAvoidanceEnv

In [4]:
from utils.reader import get_yaml_path, read_file

In [5]:
from agent.scala_dqagent import DQAgent
from training.dqnetwork import DQNetwork

2025-11-11 18:44:49.239679: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-11 18:44:49.272598: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-11 18:44:52.080647: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [6]:
from evaluation.agent_evaluation import evaluate

In [7]:
server_address = "localhost:50060" # adjust the port as needed
client_name = "RLClient"
env = ObstacleAvoidanceEnv(server_address, client_name) # adjust the env as needed
env.connect_to_client()

2025-11-11 18:44:52,933 — INFO — ✓ Connected to localhost:50060



In [8]:
configs_path = get_yaml_path("src", "scripts", "resources", "generated", "dq-obstacle-avoidance-test")
yml_files = list(configs_path.glob("*.yml"))
configs = [read_file(f) for f in yml_files]
config = configs[0]
env.init(config)

(True, '')

In [9]:
neuron_count_per_hidden_layer = [64, 32]

In [10]:
agent_id = "00000000-0000-0000-0000-000000000001"
agent = DQAgent(
    env,
    agent_id=agent_id,
    action_model=DQNetwork(
        env.observation_space.shape,
        neuron_count_per_hidden_layer,
        env.action_space.n,
        summary=False,
    ),
    target_model=DQNetwork(
        env.observation_space.shape,
        neuron_count_per_hidden_layer,
        env.action_space.n,
        summary=False,
    ),
)
agents = { agent_id: agent }

2025-11-11 18:44:53.070555: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [11]:
print(agent)

In [13]:
path = get_yaml_path("src", "scripts", "resources", "generated", "checkpoints", "deep", "high_penalty", "oa_final")
for agent in agents.values():
    agent.load(path)

/nix/store/pwrhqv9fy9lbybjz9c1w283z234lzksr-python3-3.13.8-env/lib/python3.13/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [14]:
did_succeed=lambda reward, termination, truncation: True if truncation and reward > -500 else False

In [ ]:
success_rate, median_steps_to_success = evaluate(env=env, agents=agents, configs=configs, max_steps=5000, did_succeed=did_succeed)

Evaluation:  80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                   | 8/10 [00:48<00:13,  6.71s/configuration run]

In [ ]:
print(f"success_rate:             {success_rate}")
print(f"median steps to success:  {median_steps_to_success}")